In [1]:
%pip install selenium
%pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests, urllib, re, io, time, csv, glob
import gzip
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())

url = "https://www.bseindia.com/corporates/List_Scrips.aspx"
driver.get(url)

segment = "Equity"
status = "Active"

segment_element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ddSegment"]')
segment_element.send_keys(segment)

status_element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ddlStatus"]')
status_element.send_keys(status)

submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
submit.send_keys(Keys.RETURN)


download = driver.find_element_by_xpath(
    '/html/body/div[1]/form/div[4]/div/div/div[2]/div/div/div[2]/a/i')
download.click()
time.sleep(10)
driver.quit()

df_equity = pd.read_csv("C:/Users/Sai Koteswara Rao Ch/Downloads/Equity.csv")
security_numbers = df_equity["Security Code"].tolist()
security_n = df_equity["Security Name"].tolist()
security_names = dict(zip(security_numbers, security_n))

store_cache = {}

def get_stocks(nums):
    
    def get_date(d,m,y):
        month = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[1]')   
        for opt in month.find_elements_by_tag_name("option"):
            if opt.text == m:
                opt.click()
                break
        year = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[2]')   
        for opt in year.find_elements_by_tag_name("option"):
            if opt.text == y:
                opt.click()
                break
        days = driver.find_element_by_xpath('/html/body/div[1]/table')
        for row in days.find_elements_by_tag_name('tr'):
            for col in row.find_elements_by_tag_name('td'):
                if col.text == d:
                    row.click()
                    return

    def traverse_stock(security):
        if security in store_cache:
            return None
        print(security, type(security))
        element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_smartSearch"]')
        element.clear()
        element.send_keys(security)
        element.send_keys(Keys.ENTER)
        time.sleep(10)
        try:
            response = driver.find_element_by_xpath("/html/body/form/div[4]/div/div/div[1]/div/div[3]/div/div/table/tbody/tr[2]/td/div/div[2]/div/div/div[2]/div[1]/table/tbody/tr/td/div/div/div/ul/li")
        except Exception as e:
            pass
        
        if response.text == "No Match Found":
            return None
        else:
            from_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtFromDate"]')
            from_date.clear()
            from_date.click()
            get_date("31","Dec","2015")
            to_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtToDate"]')
            to_date.clear()
            to_date.click()
            get_date("31","Dec","2020")
            submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
            submit.click()
            time.sleep(5)
            store_stock(security)
            store_cache[security] = True


    def store_stock(security):
        table = "/html/body/form/div[4]/div/div/div[1]/div/div[4]/div/div/div[1]/div[4]/div[1]/table/tbody/tr/td/div/span/table"
        table = driver.find_element_by_xpath(table)
        elements = table.find_elements_by_tag_name("tr")
        security = str(security_names[int(security)]) + "_" + str(security)+"_stocks" + ".csv"
        with open(security, "w", newline="") as csvfile:
            writer = csv.writer(csvfile)
            heading = ["Date","Open","High","Low","Close","WAP", "No. of Shares","No. of Trades","Total Turnover","Deliverable Quantity","% Deli. Qty to Traded Qty","Spread H-L","Spread C-O"]
            writer.writerow(heading)
            for row in elements[2:]:
                writer.writerow(
                    [data.text for data in row.find_elements_by_tag_name("td")])
                
    print(nums)
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument("--headless")
    driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chromeOptions)
    url = "https://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.aspx?flag=0"
    driver.get(url)
    traverse_stock(str(nums))
    driver.quit()

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Sai Koteswara Rao Ch\.wdm\drivers\chromedriver\win32\88.0.4324.96]
